<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Separate-Date-Column" data-toc-modified-id="Separate-Date-Column-1">Separate Date Column</a></span></li><li><span><a href="#Same-Day-Raw-Returns" data-toc-modified-id="Same-Day-Raw-Returns-2">Same Day Raw Returns</a></span></li><li><span><a href="#Next-Day-Raw-Return" data-toc-modified-id="Next-Day-Raw-Return-3">Next Day Raw Return</a></span></li><li><span><a href="#Average-Return-of-S&amp;P-500-Stocks" data-toc-modified-id="Average-Return-of-S&amp;P-500-Stocks-4">Average Return of S&amp;P 500 Stocks</a></span></li><li><span><a href="#Same-Day-Relative-Returns" data-toc-modified-id="Same-Day-Relative-Returns-5">Same Day Relative Returns</a></span></li><li><span><a href="#Next-Day-Relative-Returns" data-toc-modified-id="Next-Day-Relative-Returns-6">Next Day Relative Returns</a></span></li><li><span><a href="#Iterate-Through-Stocks" data-toc-modified-id="Iterate-Through-Stocks-7">Iterate Through Stocks</a></span></li></ul></div>

In [47]:
import pandas as pd
import numpy as np
import os
import time

## Separate Date Column

In [48]:
df = pd.read_csv("../00-data/stock_data/AAPL.csv", usecols=["Date", "Open", "Close"])
df

,Date,Open,Close
0,2021-01-04,133.520004,129.410004
1,2021-01-05,128.889999,131.009995
2,2021-01-06,127.720001,126.599998
3,2021-01-07,128.360001,130.919998
4,2021-01-08,132.429993,132.050003
...,...,...,...
246,2021-12-23,175.850006,176.279999
247,2021-12-27,177.089996,180.330002
248,2021-12-28,180.160004,179.289993
249,2021-12-29,179.330002,179.380005


In [49]:
year_col = []
month_col = []
day_col = []
for i in range(len(df)):   
    date_col = df.Date[i].split("-")
    year_col.append(date_col[0])
    month_col.append(date_col[1])
    day_col.append(date_col[2])
    
df['year'] = year_col
df['month'] = month_col
df['day'] = day_col
df

,Date,Open,Close,year,month,day
0,2021-01-04,133.520004,129.410004,2021,01,04
1,2021-01-05,128.889999,131.009995,2021,01,05
2,2021-01-06,127.720001,126.599998,2021,01,06
3,2021-01-07,128.360001,130.919998,2021,01,07
4,2021-01-08,132.429993,132.050003,2021,01,08
...,...,...,...,...,...,...
246,2021-12-23,175.850006,176.279999,2021,12,23
247,2021-12-27,177.089996,180.330002,2021,12,27
248,2021-12-28,180.160004,179.289993,2021,12,28
249,2021-12-29,179.330002,179.380005,2021,12,29


## Same Day Raw Returns

In [50]:
df["same_day_raw"] = ((df.Open - df.Close) / df.Open)*100
df

,Date,Open,Close,year,month,day,same_day_raw
0,2021-01-04,133.520004,129.410004,2021,01,04,3.078190
1,2021-01-05,128.889999,131.009995,2021,01,05,-1.644810
2,2021-01-06,127.720001,126.599998,2021,01,06,0.876921
3,2021-01-07,128.360001,130.919998,2021,01,07,-1.994388
4,2021-01-08,132.429993,132.050003,2021,01,08,0.286937
...,...,...,...,...,...,...,...
246,2021-12-23,175.850006,176.279999,2021,12,23,-0.244523
247,2021-12-27,177.089996,180.330002,2021,12,27,-1.829582
248,2021-12-28,180.160004,179.289993,2021,12,28,0.482910
249,2021-12-29,179.330002,179.380005,2021,12,29,-0.027883


## Next Day Raw Return

In [51]:
df["next_day_raw"] = list(df.same_day_raw[1:]) + [np.nan] 
df

,Date,Open,Close,year,month,day,same_day_raw,next_day_raw
0,2021-01-04,133.520004,129.410004,2021,01,04,3.078190,-1.644810
1,2021-01-05,128.889999,131.009995,2021,01,05,-1.644810,0.876921
2,2021-01-06,127.720001,126.599998,2021,01,06,0.876921,-1.994388
3,2021-01-07,128.360001,130.919998,2021,01,07,-1.994388,0.286937
4,2021-01-08,132.429993,132.050003,2021,01,08,0.286937,0.162556
...,...,...,...,...,...,...,...,...
246,2021-12-23,175.850006,176.279999,2021,12,23,-0.244523,-1.829582
247,2021-12-27,177.089996,180.330002,2021,12,27,-1.829582,0.482910
248,2021-12-28,180.160004,179.289993,2021,12,28,0.482910,-0.027883
249,2021-12-29,179.330002,179.380005,2021,12,29,-0.027883,0.707641


## Average Return of S&P 500 Stocks

In [52]:
daily_avg_return = pd.DataFrame(df.groupby(["year", "month", "day"])["same_day_raw"]
                                .mean()).reset_index().rename(columns={"same_day_raw":"daily_avg_raw"})
daily_avg_return

,year,month,day,daily_avg_raw
0,2021,01,04,3.078190
1,2021,01,05,-1.644810
2,2021,01,06,0.876921
3,2021,01,07,-1.994388
4,2021,01,08,0.286937
...,...,...,...,...
246,2021,12,23,-0.244523
247,2021,12,27,-1.829582
248,2021,12,28,0.482910
249,2021,12,29,-0.027883


In [53]:
df = df.merge(daily_avg_return, on = ["year", "month", "day"])
df

,Date,Open,Close,year,month,day,same_day_raw,next_day_raw,daily_avg_raw
0,2021-01-04,133.520004,129.410004,2021,01,04,3.078190,-1.644810,3.078190
1,2021-01-05,128.889999,131.009995,2021,01,05,-1.644810,0.876921,-1.644810
2,2021-01-06,127.720001,126.599998,2021,01,06,0.876921,-1.994388,0.876921
3,2021-01-07,128.360001,130.919998,2021,01,07,-1.994388,0.286937,-1.994388
4,2021-01-08,132.429993,132.050003,2021,01,08,0.286937,0.162556,0.286937
...,...,...,...,...,...,...,...,...,...
246,2021-12-23,175.850006,176.279999,2021,12,23,-0.244523,-1.829582,-0.244523
247,2021-12-27,177.089996,180.330002,2021,12,27,-1.829582,0.482910,-1.829582
248,2021-12-28,180.160004,179.289993,2021,12,28,0.482910,-0.027883,0.482910
249,2021-12-29,179.330002,179.380005,2021,12,29,-0.027883,0.707641,-0.027883


## Same Day Relative Returns

In [54]:
df["same_day_relative"] = df.same_day_raw - df.daily_avg_raw
df

,Date,Open,Close,year,month,day,same_day_raw,next_day_raw,daily_avg_raw,same_day_relative
0,2021-01-04,133.520004,129.410004,2021,01,04,3.078190,-1.644810,3.078190,0.0
1,2021-01-05,128.889999,131.009995,2021,01,05,-1.644810,0.876921,-1.644810,0.0
2,2021-01-06,127.720001,126.599998,2021,01,06,0.876921,-1.994388,0.876921,0.0
3,2021-01-07,128.360001,130.919998,2021,01,07,-1.994388,0.286937,-1.994388,0.0
4,2021-01-08,132.429993,132.050003,2021,01,08,0.286937,0.162556,0.286937,0.0
...,...,...,...,...,...,...,...,...,...,...
246,2021-12-23,175.850006,176.279999,2021,12,23,-0.244523,-1.829582,-0.244523,0.0
247,2021-12-27,177.089996,180.330002,2021,12,27,-1.829582,0.482910,-1.829582,0.0
248,2021-12-28,180.160004,179.289993,2021,12,28,0.482910,-0.027883,0.482910,0.0
249,2021-12-29,179.330002,179.380005,2021,12,29,-0.027883,0.707641,-0.027883,0.0


## Next Day Relative Returns

In [55]:
df["next_day_relative"] = list(df.same_day_relative[1:]) + [np.nan] 
df

,Date,Open,Close,year,month,day,same_day_raw,next_day_raw,daily_avg_raw,same_day_relative,next_day_relative
0,2021-01-04,133.520004,129.410004,2021,01,04,3.078190,-1.644810,3.078190,0.0,0.0
1,2021-01-05,128.889999,131.009995,2021,01,05,-1.644810,0.876921,-1.644810,0.0,0.0
2,2021-01-06,127.720001,126.599998,2021,01,06,0.876921,-1.994388,0.876921,0.0,0.0
3,2021-01-07,128.360001,130.919998,2021,01,07,-1.994388,0.286937,-1.994388,0.0,0.0
4,2021-01-08,132.429993,132.050003,2021,01,08,0.286937,0.162556,0.286937,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
246,2021-12-23,175.850006,176.279999,2021,12,23,-0.244523,-1.829582,-0.244523,0.0,0.0
247,2021-12-27,177.089996,180.330002,2021,12,27,-1.829582,0.482910,-1.829582,0.0,0.0
248,2021-12-28,180.160004,179.289993,2021,12,28,0.482910,-0.027883,0.482910,0.0,0.0
249,2021-12-29,179.330002,179.380005,2021,12,29,-0.027883,0.707641,-0.027883,0.0,0.0


## Iterate Through Stocks

In [56]:
stock_csvs = os.listdir("../00-data/stock_data/")
stock_csvs[0:5]

['CSCO.csv', 'UAL.csv', 'TROW.csv', 'ISRG.csv', 'NVR.csv']

In [57]:
tickers = [t.replace(".csv", "") for t in stock_csvs]
tickers[0:5]

['CSCO', 'UAL', 'TROW', 'ISRG', 'NVR']

In [58]:
# time process
tic = time.perf_counter()

In [59]:
final_df = pd.DataFrame()
bad_files = []
for t in tickers:
    print(t)
    # read file in
    path = "../00-data/stock_data/"
    end = ".csv"
    file_path = path + t + end
    try:
        df = pd.read_csv(file_path, usecols=["Date", "Open", "Close"])
    except FileNotFoundError:
        bad_files.append(t)
        continue
    
    # separate date cols
    year_col = []
    month_col = []
    day_col = []
    for i in range(len(df)):   
        date_col = df.Date[i].split("-")
        year_col.append(date_col[0])
        month_col.append(date_col[1])
        day_col.append(date_col[2])
    df['year'] = year_col
    df['month'] = month_col
    df['day'] = day_col
    df = df.drop("Date",axis=1)
    
    # add ticker col
    tick = [t] * len(df)
    df['ticker'] = tick
    
    # same day raw returns
    df["same_day_raw"] = ((df.Open - df.Close) / df.Open)*100
    
    # next day raw returns
    df["next_day_raw"] = list(df.same_day_raw[1:]) + [np.nan] 
    df = df.drop(["Open","Close"],axis=1)
    
    # concat to final df
    final_df = pd.concat([final_df,df], axis=0)

# avg s&p 500 returns
sp_avg_return = pd.DataFrame(final_df.groupby(["year", "month", "day"])["same_day_raw"]
                                .mean()).reset_index().rename(columns={"same_day_raw":"sp_avg_return"})
final_df = final_df.merge(sp_avg_return, on = ["year", "month", "day"])

# same day relative returns
final_df["same_day_relative"] = final_df.same_day_raw - final_df.sp_avg_return

# next day raw returns 
final_df["next_day_relative"] = list(final_df.same_day_relative[1:]) + [np.nan] 

CSCO
UAL
TROW
ISRG
NVR
TPR
DVN
CE
MRO
BA
VRTX
GILD
NLSN
EQIX
TER
MDT
V
QRVO
A
FOX
FLT
MO
CTRA
SWKS
ENPH
MCHP
CDNS
MSCI
CHTR
EIX
BBY
WBA
LVS
HCA
AJG
DTE
C
T
CF
DISH
MGM
HUM
CBOE
CFG
APH
SYY
MSI
FCX
ADM
OGN
LH
PKI
LNT
BAC
LNC
PSX
GPN
PPG
TECH
IRM
IQV
ESS
WBD
HAL
STZ
DXC
PARA
ADI
F
ADBE
.DS_Store
CPRT
TDG
TFX
ULTA
ARE
SYK
CB
TSN
GNRC
PEP
PEG
NOW
LLY
COST
REG
NWS
LOW
MDLZ
BKNG
ZBRA
FMC
XEL
AIZ
CERN
MET
FTV
DLR
XRAY
FAST
TJX
SNA
MPC
BR
D
MRK
STX
NOC
BXP
KHC
IPG
UNP
ALLE
ABBV
CDAY
ORCL
ECL
ETR
EBAY
SBUX
PENN
IR
AMT
INTU
DPZ
PAYC
DRE
CMA
IPGP
PG
CAT
ODFL
MCD
MNST
AMZN
INTC
PNR
GLW
BDX
KMI
PWR
APTV
BBWI
DXCM
EXR
WELL
HOLX
EXPD
GM
TXN
VRSK
SJM
TMO
OXY
RL
CCI
MMM
MOS
FTNT
HSY
JNPR
DHI
ED
ES
ADSK
GL
IP
EXPE
KO
PCAR
WDC
LUMN
PYPL
NEE
UPS
EMR
MSFT
ANSS
CTAS
BIO
UDR
CTLT
WEC
AME
IT
DD
ACN
VRSN
EW
CMG
AWK
COO
SHW
HPQ
AMAT
CCL
MLM
AVY
AAP
ATVI
EVRG
EA
DE
SPG
AMD
KLAC
NDAQ
URI
WHR
RTX
NXPI
PNC
KMX
SEDG
WRK
MTCH
BIIB
NVDA
CHRW
ROP
IDXX
EXC
HES
HD
ALB
VLO
AON
ZTS
FDX
DG
TYL
HIG
CMS
CAG
INCY
SCHW
HSIC
AZO

In [60]:
final_df

,year,month,day,ticker,same_day_raw,next_day_raw,sp_avg_return,same_day_relative,next_day_relative
0,2021,01,04,CSCO,0.789894,-0.686813,1.933110,-1.143216,2.760111
1,2021,01,04,UAL,4.693221,-4.377104,1.933110,2.760111,1.068330
2,2021,01,04,TROW,3.001440,-1.030012,1.933110,1.068330,1.195829
3,2021,01,04,ISRG,3.128939,-1.405737,1.933110,1.195829,-0.726990
4,2021,01,04,NVR,1.206120,0.404701,1.933110,-0.726990,-4.024240
...,...,...,...,...,...,...,...,...,...
127914,2021,12,30,LRCX,1.328294,NaN,0.254755,1.073540,0.389219
127915,2021,12,30,NWL,0.643974,NaN,0.254755,0.389219,0.265335
127916,2021,12,30,UAA,0.520090,NaN,0.254755,0.265335,-0.478512
127917,2021,12,30,BLK,-0.223757,NaN,0.254755,-0.478512,-0.455498


In [61]:
final_df.shape

(127919, 9)

In [65]:
final_df.to_csv("../00-data/stock_returns.csv")

In [63]:
toc = time.perf_counter()
print(f"Appended in {(toc - tic)/60:0.2f} minutes")

Appended in 0.22 minutes
